In [108]:
#https://preppindata.blogspot.com/2021/01/2021-week-4.html

import pandas as pd

### Input the file Union the Stores data together

In [92]:
df = pd.concat(pd.read_excel(r'data/PD 2021 Wk 4 Input.xlsx',sheet_name=None)).reset_index()
df.head(5)

,level_0,level_1,Date,New - Saddles,New - Mudguards,New - Wheels,New - Bags,Existing - Saddles,Existing - Mudguards,Existing - Wheels,Existing - Bags,Quarter,Store,Target
0,Manchester,0,2021-01-21,13.0,42.0,19.0,38.0,17.0,48.0,19.0,13.0,NaN,NaN,NaN
1,Manchester,1,2021-02-21,1.0,9.0,14.0,6.0,2.0,4.0,19.0,24.0,NaN,NaN,NaN
2,Manchester,2,2021-03-21,8.0,22.0,6.0,35.0,0.0,48.0,17.0,16.0,NaN,NaN,NaN
3,Manchester,3,2021-04-21,3.0,9.0,8.0,16.0,18.0,50.0,18.0,25.0,NaN,NaN,NaN
4,Manchester,4,2021-05-21,2.0,8.0,5.0,34.0,17.0,3.0,12.0,19.0,NaN,NaN,NaN


Remove any unnecessary data fields your Input step might create and rename the 'Table Names' as 'Store' 

In [93]:
df = df.drop(['level_1', 'Quarter','Store','Target'],axis=1)
df.rename(columns={'level_0': 'Store'},inplace=True)

df.head(5)

,Store,Date,New - Saddles,New - Mudguards,New - Wheels,New - Bags,Existing - Saddles,Existing - Mudguards,Existing - Wheels,Existing - Bags
0,Manchester,2021-01-21,13.0,42.0,19.0,38.0,17.0,48.0,19.0,13.0
1,Manchester,2021-02-21,1.0,9.0,14.0,6.0,2.0,4.0,19.0,24.0
2,Manchester,2021-03-21,8.0,22.0,6.0,35.0,0.0,48.0,17.0,16.0
3,Manchester,2021-04-21,3.0,9.0,8.0,16.0,18.0,50.0,18.0,25.0
4,Manchester,2021-05-21,2.0,8.0,5.0,34.0,17.0,3.0,12.0,19.0


### Pivot the product columns & rename the Values column resulting from you pivot as 'Products Sold'

In [94]:
df_pivot = df.melt(id_vars=['Store','Date'],
        var_name='Customer Type - Product',
        value_name='Products Sold')
df_pivot.head(5)

,Store,Date,Customer Type - Product,Products Sold
0,Manchester,2021-01-21,New - Saddles,13.0
1,Manchester,2021-02-21,New - Saddles,1.0
2,Manchester,2021-03-21,New - Saddles,8.0
3,Manchester,2021-04-21,New - Saddles,3.0
4,Manchester,2021-05-21,New - Saddles,2.0


### Split the 'Customer Type - Product' field to create: 

In [95]:
df_pivot[['Customer Type','Product'] ] = df_pivot['Customer Type - Product'].str.split(' - ',expand=True)
df_pivot

,Store,Date,Customer Type - Product,Products Sold,Customer Type,Product
0,Manchester,2021-01-21,New - Saddles,13.0,New,Saddles
1,Manchester,2021-02-21,New - Saddles,1.0,New,Saddles
2,Manchester,2021-03-21,New - Saddles,8.0,New,Saddles
3,Manchester,2021-04-21,New - Saddles,3.0,New,Saddles
4,Manchester,2021-05-21,New - Saddles,2.0,New,Saddles
...,...,...,...,...,...,...
635,Targets,NaT,Existing - Bags,NaN,Existing,Bags
636,Targets,NaT,Existing - Bags,NaN,Existing,Bags
637,Targets,NaT,Existing - Bags,NaN,Existing,Bags
638,Targets,NaT,Existing - Bags,NaN,Existing,Bags


### Turn the date into a 'Quarter' number 

In [96]:
df_pivot['Quarter'] = df_pivot['Date'].dt.quarter

### Sum up the products sold by Store and Quarter

In [97]:
df_grouped = df_pivot.groupby(['Store', 'Quarter'],as_index=False)['Products Sold'].agg('sum')

df_grouped

,Store,Quarter,Products Sold
0,Birmingham,1.0,477.0
1,Birmingham,2.0,346.0
2,Birmingham,3.0,348.0
3,Birmingham,4.0,404.0
4,Leeds,1.0,488.0
5,Leeds,2.0,331.0
6,Leeds,3.0,279.0
7,Leeds,4.0,349.0
8,London,1.0,425.0
9,London,2.0,324.0


### Add the Targets data & Join the Targets data with the aggregated Stores data 

In [98]:
df_target = pd.read_csv(r'data/PD 2021 Wk 4 input - target.csv')
df_output = df_target.merge(df_grouped, on=['Store','Quarter'], how='inner')

### Remove any duplicate fields formed by the Join

In [99]:
df_output = df_output.drop_duplicates()

### Calculate the Variance between each Store's Quarterly actual sales and the target. Call this field 'Variance to Target'

In [106]:
df_output['Variance to Target'] = df_output['Products Sold'] - df_output['Target']
df_output.head()

,Quarter,Store,Target,Products Sold,Variance to Target,Rank
0,2,York,300,329.0,29.0,5
1,2,Birmingham,325,346.0,21.0,4
2,2,Leeds,325,331.0,6.0,3
3,2,London,325,324.0,-1.0,2
4,2,Manchester,300,288.0,-12.0,1


### Rank the Store's based on the Variance to Target in each quarter (The greater the variance the better the rank)

In [105]:
#new column rank grouped by quarter, change into integer
df_output['Rank'] = df_output.groupby(['Quarter'])['Variance to Target'].rank().astype(int)
df_output.head()

,Quarter,Store,Target,Products Sold,Variance to Target,Rank
0,2,York,300,329.0,29.0,5
1,2,Birmingham,325,346.0,21.0,4
2,2,Leeds,325,331.0,6.0,3
3,2,London,325,324.0,-1.0,2
4,2,Manchester,300,288.0,-12.0,1


### Output the data

In [107]:
df_output.to_csv(r'output/2021-week4-output.csv')